In [7]:
from torchvision.datasets import ImageFolder
import basic_torch_function
from torchvision import models
path = "D:\\Casper\\OTHER\\Data\\rock_paper_scissor\\rps-cv-images"
# path = "D:\\Casper\\OTHER\\Data\\dpaml_hw3\\jpg_dataset"
path = "D:\\Casper\\OTHER\\Data\\dpaml_hw3\\face_data"
# path = "D:\\Casper\\OTHER\\Data\\rock_paper_scissor\\rps-cv-images"
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import ConcatDataset

# from torchvision.transforms import v2
# transforms = v2.Compose([
#     v2.RandomResizedCrop(size=(224, 224), antialias=True),
#     v2.ToDtype(torch.float32, scale=True),
# ])
img_size = [512, 512]
# img_size = [512, 512]
transforming = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
])

transforming0 = transforms.Compose([
    transforms.Resize(img_size),
    transforms.RandomVerticalFlip(1),
    transforms.ToTensor(),
])

transforming1 = transforms.Compose([
    transforms.Resize(img_size),
    transforms.RandomHorizontalFlip(1),
    transforms.ToTensor(),
])

transforming2 = transforms.Compose([
    transforms.Resize(img_size),
    transforms.RandomHorizontalFlip(1),
    transforms.RandomVerticalFlip(1),
    transforms.ToTensor(),
])

transforming3 = transforms.Compose([
    transforms.Resize(img_size),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.ToTensor(),
])

dataset = ImageFolder(path, transforming)
dataset_a0 = ImageFolder(path, transforming0)
dataset_a1 = ImageFolder(path, transforming1)
dataset_a2 = ImageFolder(path, transforming2)
merge_dataset = ConcatDataset([
    dataset,
    dataset_a0, 
    dataset_a1, 
    dataset_a2, 
    ])

In [8]:
data_loaders = basic_torch_function.get_dataloaders(merge_dataset, 0.8, 1, 4)

--------- INDEX checking ---------
Original: [0, 1, 2, 3, 4]
Shuffled: [216, 713, 807, 854, 393]
--------- INDEX shuffled ---------

Total number of samples: 1240 datapoints
Number of train samples: 248 batches/ 992 datapoints
Number of val samples: 62 batches/ 248 datapoints
Number of test samples: 0 batches/ 0 datapoints



In [9]:
class SimpleCNN_a(nn.Module):
    def __init__(self):
        super(SimpleCNN_a, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, (3,3), 2)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(20, 40, (3,3), 2)
        self.conv3 = nn.Conv2d(40, 20, (3,3), 2)
        self.conv4 = nn.Conv2d(20, 1, (3,3), 2)
        self.fc = nn.Linear(1*961, 4)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        # x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        # x = self.pool(x)
        x = self.conv3(x)
        x = F.relu(x)
        # x = self.pool(x)
        x = self.conv4(x)
        x = F.relu(x)
        # x = self.pool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x
    
class SimpleCNN_b(nn.Module):
    def __init__(self):
        super(SimpleCNN_b, self).__init__()
        self.conv1 = nn.Conv2d(3, 50, (3,3), 2)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(50, 1, (3,3), 2)
        self.fc = nn.Linear(1*961, 4)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x
model = SimpleCNN_b()
img = torch.rand(2,3,512,512)
model(img).shape

torch.Size([2, 4])

In [10]:
basic_torch_function.pprint("======================")
basic_torch_function.pprint("--- Start Training ---")
basic_torch_function.pprint("======================")

--- Start Training ---


In [11]:
print(models.resnet18())

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
model_list ={
    "resnet18": lambda: models.resnet18(),
    "mobileNet_v2": lambda: models.mobilenet_v2(),
    # "resnet18_mod": lambda: mod_resnet(BasicBlock, [2, 2, 2, 2], channel_num_list=[2, 2, 2, 2, 2], num_classes=3, input_channel=3),
    # "resnet18_mod2": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[2, 2, 2, 2, 2], num_classes=3, input_channel=3),
    # "mod_face": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[64, 64, 64, 64, 64], num_classes=3, input_channel=3),
    # "mod_face1": lambda: mod_resnet(BasicBlock, [2, 2, 2, 2], channel_num_list=[8, 8, 16, 16, 8], num_classes=3, input_channel=3),
    # "mod_face2": lambda: mod_resnet(BasicBlock, [2, 2, 2, 2], channel_num_list=[16, 16, 32, 64, 128], num_classes=3, input_channel=3),
    # "mod_face3": lambda: mod_resnet(BasicBlock, [2, 4, 4, 2], channel_num_list=[8, 8, 8, 8, 8], num_classes=3, input_channel=3),
    # "mod_face4": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[4, 4, 4, 4, 4], num_classes=3, input_channel=3),
    # "mod_face4": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[4, 4, 4, 4, 4], num_classes=3, input_channel=3),
    "mod_face_a": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[4, 16, 16, ], num_classes=4, input_channel=3),
    "mod_face_b": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[4, 8, 8, ], num_classes=4, input_channel=3),
    # "mod_face5": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[16, 16, 16, 8, 8], num_classes=3, input_channel=3),
    "SimpleCNN_a": lambda: SimpleCNN_a(),
    "SimpleCNN_b": lambda: SimpleCNN_b(),
    # "resnet18_pre": lambda: models.resnet18(weights=models.ResNet18_Weights.DEFAULT),
}
model_names = [

    "resnet18", # 11,178,051
    "mobileNet_v2", # 2,227,715
    "mod_face_a", # 17,991
    "mod_face_b", # 5,199
    "SimpleCNN_a", # 19,049
    "SimpleCNN_b", # 4,737
]

for ii in range(len(model_names)):
    model_name = model_names[ii]
    phases = ['train', 'val']
    basic_torch_function.train(model_list, model_name, data_loaders, 4, phases, epochs=25, save_weight=True)

test mod_face_a
model total parameters: 17,991
learning rate=0.005


100%|██████████| 875/875 [00:27<00:00, 32.20it/s]


Epoch [1/25], phase: train, samples: 3500, Loss: 0.2046, Top-1 Accuracy: 62.80%, Top-3 Accuracy: 100.00%


100%|██████████| 219/219 [00:04<00:00, 49.23it/s]


Epoch [1/25], phase: val, samples: 876, Loss: 0.1823, Top-1 Accuracy: 77.05%, Top-3 Accuracy: 100.00%


...

Epoch [2/25], phase: val, samples: 876, Loss: 0.3693, Top-1 Accuracy: 61.30%, Top-3 Accuracy: 100.00%


100%|██████████| 875/875 [00:17<00:00, 49.48it/s]


Epoch [3/25], phase: train, samples: 3500, Loss: 0.0670, Top-1 Accuracy: 91.46%, Top-3 Accuracy: 100.00%


 56%|█████▌    | 122/219 [00:02<00:01, 53.76it/s]


KeyboardInterrupt: 

In [11]:
import torch
model = torch.jit.load('mod_face_a.pt')
print(model)
model.eval()

RecursiveScriptModule(
  original_name=mod_resnet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
  (layer1): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      (downsample): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=Conv2d)
        (1): RecursiveScriptModule(original_name=BatchNorm2d)
      )
    )
    (1): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv2): 

RecursiveScriptModule(
  original_name=mod_resnet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
  (layer1): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
      (downsample): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=Conv2d)
        (1): RecursiveScriptModule(original_name=BatchNorm2d)
      )
    )
    (1): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (conv2): 

In [18]:
for images, labels in data_loaders['train']:
    train_images = images.view(images.size(0), -1).numpy()  # Flatten the images
    train_labels = labels.numpy()
    break

for images, labels in data_loaders['val']:
    val_images = images.view(images.size(0), -1).numpy()  # Flatten the images
    val_labels = labels.numpy()
    break

In [19]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
# Standardizing and applying PCA
# Using a pipeline to chain PCA and SVM
pipe = make_pipeline(StandardScaler(), PCA(n_components=10), SVC(gamma='auto'))
# for images, labels in tqdm(loaders[phase]): # Iterate over data.
print("starting to fit")
# Fit and train in one step
pipe.fit(train_images, train_labels)
print("finish fitting")
val_predictions = pipe.predict(val_images)
val_accuracy = accuracy_score(val_labels, val_predictions)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print("The model is trained and ready to make predictions.")


starting to fit
finish fitting
Validation Accuracy: 35.29%
The model is trained and ready to make predictions.


In [20]:
train_predictions = pipe.predict(train_images)
train_accuracy = accuracy_score(train_labels, train_predictions)

print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print("The model is trained and ready to make predictions.")

Train Accuracy: 100.00%
The model is trained and ready to make predictions.


In [ ]:
def one_hot_decode(data):
    data = np.squeeze(data)
    list_max=data.tolist()
    list_max.sort(key=lambda x: float(x), reverse = True)
    n = np.argwhere(data==list_max[0])
    return int(n), list_max[0]
def translate_Y(yi , show = False):
    
    num = {
        0 : "good", 1 : "other", 2 : "paper", 3 : "rock", 4 : "scissor", } 
    
    yi_new = num.get( yi ) 
            
    if show:
        print(yi)
        print(yi_new)            
        
    return yi_new
model = models.load_model(name)
print("load ",name)

# 選擇第攝影機
cap = cv2.VideoCapture(0)

In [ ]:
while(True):
    # 從攝影機擷取一張影像
    ret, frame = cap.read()
    
    # 模型預估
    frame_resize = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)
    input_data=frame_resize[np.newaxis,:,:,:]
    start = time.time()
    prediction=model.predict(input_data) 
    end = time.time()
    accumulate_time = (end - start)
    FPS = 1/accumulate_time
    FPS = round(FPS,2)
    result = one_hot_decode(prediction)
    detected = translate_Y(result[0])
    #print(detected)
    
    # 顯示圖片
    cv2.putText(frame, str(FPS), (540, 40), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.putText(frame, detected, (10, 40), cv2.FONT_HERSHEY_SIMPLEX,1, (255, 0, 0), 1, cv2.LINE_AA)
    cv2.imshow('frame', frame)
    
    # 若按下 q 鍵則離開迴圈
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv2.destroyAllWindows()


# 越深越容易不穩定？？

In [4]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = F.relu(out)

        return out
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x
class mod_resnet(nn.Module):
    def __init__(self, block, layers, channel_num_list, num_classes=1000, input_channel=3):
        super(mod_resnet, self).__init__()
        self.in_channels = channel_num_list[0]
        self.conv1 = nn.Conv2d(input_channel, channel_num_list[0], kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_num_list[0])
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, channel_num_list[1], layers[0])
        self.layer2 = self._make_layer(block, channel_num_list[2], layers[1], stride=2)
        # self.layer3 = self._make_layer(block, channel_num_list[3], layers[2], stride=2)
        # self.layer4 = self._make_layer(block, channel_num_list[4], layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(channel_num_list[-1] * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        # x = self.layer3(x)
        # x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x